<a href="https://colab.research.google.com/github/dakilaledesma/Atomized_NN_Experiments/blob/main/resolution_differences/GPU_Sequencer2D_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 151 ms, sys: 20.1 ms, total: 171 ms
Wall time: 23.3 s


In [2]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10776, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 10776 (delta 161), reused 223 (delta 123), pack-reused 10473
Receiving objects: 100% (10776/10776), 20.47 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (7877/7877), done.


In [3]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model sequencer2d_l --opt adabelief --lr 0.0001 --epochs 80 --aa v0r-mstd0.5 --aug-splits 2 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 224 224 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/ 

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth)
Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth
Model sequencer2d_l created, param count:54028716
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225